In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import seaborn as sns

In [2]:
data_dir = 'D:/NN_labs/lab4/101_food_classes_10_percent/train/'
classes = sorted(os.listdir(data_dir))
selected_classes = [classes[4], classes[34], classes[64]]

In [4]:
train_data = []
test_data = []

for clas in selected_classes:
    class_path = os.path.join(data_dir, clas)
    class_images = [os.path.join(clas, img) for img in os.listdir(class_path)]
    train, test = train_test_split(class_images, test_size=0.2, random_state=42)
    train_data.extend([(clas, img) for img in train])
    test_data.extend([(clas, img) for img in test])

train_df = pd.DataFrame(train_data, columns=['class', 'image'])
test_df = pd.DataFrame(test_data, columns=['class', 'image'])

In [6]:
img_size = (224, 224)
batch_size = 32

train_datagenerator = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_generator = train_datagenerator.flow_from_dataframe(train_df, directory=data_dir, x_col='image', y_col='class', target_size=img_size, batch_size=batch_size, class_mode='categorical')
test_generator = test_datagenerator.flow_from_dataframe(test_df, directory=data_dir, x_col='image', y_col='class', target_size=img_size, batch_size=batch_size, class_mode='categorical', shuffle=False)

Found 180 validated image filenames belonging to 3 classes.
Found 45 validated image filenames belonging to 3 classes.


In [7]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(*img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(612, activation='relu'),
    Dense(3, activation='softmax')])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
model.fit(train_generator,epochs=10,validation_data = test_generator)

Epoch 1/10
6/6 [==============================] - 15s 2s/step - loss: 3.5351 - accuracy: 0.3333 - val_loss: 1.1531 - val_accuracy: 0.3333
Epoch 2/10
6/6 [==============================] - 11s 2s/step - loss: 1.1180 - accuracy: 0.3389 - val_loss: 1.1208 - val_accuracy: 0.3333
Epoch 3/10
6/6 [==============================] - 11s 2s/step - loss: 1.0663 - accuracy: 0.4556 - val_loss: 1.0250 - val_accuracy: 0.4889
Epoch 4/10
6/6 [==============================] - 11s 2s/step - loss: 0.9833 - accuracy: 0.5111 - val_loss: 0.8845 - val_accuracy: 0.5778
Epoch 5/10
6/6 [==============================] - 11s 2s/step - loss: 0.8984 - accuracy: 0.5389 - val_loss: 0.8209 - val_accuracy: 0.6222
Epoch 6/10
6/6 [==============================] - 11s 2s/step - loss: 0.8397 - accuracy: 0.6056 - val_loss: 0.7645 - val_accuracy: 0.6444
Epoch 7/10
6/6 [==============================] - 11s 2s/step - loss: 0.7515 - accuracy: 0.6444 - val_loss: 0.8164 - val_accuracy: 0.6222
Epoch 8/10
6/6 [==================

In [9]:
loss, accuracy = model.evaluate(test_generator, verbose=0)
print('Test loss: ', round(loss, 2))
print('Test accuracy: ', round(accuracy, 2))

Test loss:  0.69
Test accuracy:  0.69
